In [ ]:
!pip install gymnasium
!pip install tetris_gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 11.5 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
from tetris_gymnasium.envs import Tetris
import matplotlib
import matplotlib.pyplot as plt

# Set up Colab

In [ ]:
env = gym.make('tetris_gymnasium/Tetris', render_mode='rgb_array')

if 'inline' in matplotlib.get_backend():
  from IPython import display
plt.ion()

# Setting up for Training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque

class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()

        self.fc1_1 = nn.Linear(24 * 18, 512)
        self.fc1_2 = nn.Linear(24 * 18, 512)
        self.fc1_3 = nn.Linear(4 * 4, 64)
        self.fc1_4 = nn.Linear(4 * 16, 64)

        self.fc2 = nn.Linear(512 + 512 + 64 + 64, 512)
        self.fc3 = nn.Linear(512, 8)

    def forward(self, board, active_tetromino_mask, holder, queue):
        x1 = torch.relu(self.fc1_1(board.view(1, 432)))
        x2 = torch.relu(self.fc1_2(active_tetromino_mask.view(1, 432)))
        x3 = torch.relu(self.fc1_3(holder.view(1, 16)))
        x4 = torch.relu(self.fc1_4(queue.view(1, 64)))

        x = torch.cat((x1, x2, x3, x4), dim=1)
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

class DQNAgent:
    def __init__(self, action_size, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, lr=1e-4, batch_size=32, memory_size=10000):
        self.action_size = action_size
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size

        self.model = DQN()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()

        self.memory = ReplayMemory(memory_size)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        board = torch.tensor(state['board'], dtype=torch.float32)
        active_tetromino_mask = torch.tensor(state['active_tetromino_mask'], dtype=torch.float32)
        holder = torch.tensor(state['holder'], dtype=torch.float32)
        queue = torch.tensor(state['queue'], dtype=torch.float32)

        q_values = self.model(board, active_tetromino_mask, holder, queue)
        return torch.argmax(q_values).item()

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = self.memory.sample(self.batch_size)
        for state, action, reward, next_state, done in batch:
            board, active_tetromino_mask, holder, queue = state.values()
            next_board, next_active_tetromino_mask, next_holder, next_queue = next_state.values()

            board_tensor = torch.tensor(board, dtype=torch.float32)
            active_tetromino_mask_tensor = torch.tensor(active_tetromino_mask, dtype=torch.float32)
            holder_tensor = torch.tensor(holder, dtype=torch.float32)
            queue_tensor = torch.tensor(queue, dtype=torch.float32)

            next_board_tensor = torch.tensor(next_board, dtype=torch.float32)
            next_active_tetromino_mask_tensor = torch.tensor(next_active_tetromino_mask, dtype=torch.float32)
            next_holder_tensor = torch.tensor(next_holder, dtype=torch.float32)
            next_queue_tensor = torch.tensor(next_queue, dtype=torch.float32)

            next_q_values = self.model(next_board_tensor, next_active_tetromino_mask_tensor, next_holder_tensor, next_queue_tensor)
            next_q_value = torch.max(next_q_values).item()

            target_q_value = reward + (1 - done) * self.gamma * next_q_value

            current_q_values = self.model(board_tensor, active_tetromino_mask_tensor, holder_tensor, queue_tensor)
            current_q_value = current_q_values[0][action]

            loss = self.loss_fn(current_q_value, torch.tensor(target_q_value, dtype=torch.float32))

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def train(self, env, num_episodes=1000):
        for e in range(num_episodes):
            state, _ = env.reset()
            done = False
            total_reward = 0

            while not done:
                action = self.act(state)
                next_state, reward, terminated, truncated, _ = env.step(action)
                done = terminated or truncated

                self.memory.push(state, action, reward, next_state, done)

                self.replay()

                state = next_state
                total_reward += reward

            print(f"Episode {e}/{num_episodes}, Total Reward: {total_reward}, Epsilon: {self.epsilon}")

# Training

In [ ]:
agent = DQNAgent(action_size=8)
agent.train(env, num_episodes=1000)

Episode 0/1000, Total Reward: 8, Epsilon: 0.9091562615825302
Episode 1/1000, Total Reward: 9, Epsilon: 0.798065677681905
Episode 2/1000, Total Reward: 10, Epsilon: 0.6866430931872001
Episode 3/1000, Total Reward: 10, Epsilon: 0.5848838636585911
Episode 4/1000, Total Reward: 9, Epsilon: 0.5344229416520513
Episode 5/1000, Total Reward: 9, Epsilon: 0.4982051627146237
Episode 6/1000, Total Reward: 10, Epsilon: 0.457510005540005
Episode 7/1000, Total Reward: 10, Epsilon: 0.41386834584198684
Episode 8/1000, Total Reward: 12, Epsilon: 0.322118930542046
Episode 9/1000, Total Reward: 9, Epsilon: 0.2943280830920294
Episode 10/1000, Total Reward: 9, Epsilon: 0.25966219297210513
Episode 11/1000, Total Reward: 12, Epsilon: 0.18373897616330553
Episode 12/1000, Total Reward: 13, Epsilon: 0.08619587539414532
Episode 13/1000, Total Reward: 11, Epsilon: 0.04251487140556204
Episode 14/1000, Total Reward: 13, Epsilon: 0.020044857891939702
Episode 15/1000, Total Reward: 10, Epsilon: 0.011094979641301777
Ep

KeyboardInterrupt: 

# Testing

In [ ]:
from pathlib import Path
import numpy as np

done = False
total_reward = 0
frames = []
state, _ = env.reset()

while not done:
  action = agent.act(state)
  next_state, reward, terminated, truncated, _ = env.step(action)
  agent.memory.push(state, action, reward, next_state, done)
  agent.replay()
  frame = env.render()
  frames.append(frame)
  done = terminated or truncated
  total_reward+=reward
  state = next_state

env.close()
print(f'Total rewards: {total_reward}')

import imageio
video_path = 'tetris_test.mp4'
imageio.mimsave(video_path, frames, fps=30)

from IPython.display import HTML
from base64 import b64encode
mp4 = open(video_path,'rb').read()
video_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width="640" height="480" controls><source src="{video_url}" type="video/mp4"></video>')